In [1]:
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI
from langchain import LLMMathChain, SerpAPIWrapper
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentType
import os
from getpass import getpass
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
import re
import ast
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

c:\Users\asimj\.conda\envs\langchain\lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (3.6.12) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [122]:
temp = ast.literal_eval(r"""{"boom": "boom"}""")
temp["boom"]

'boom'

In [15]:
from substrateinterface import SubstrateInterface, Keypair
from substrateinterface.contracts import ContractCode, ContractInstance
from substrateinterface.exceptions import SubstrateRequestException

substrate_relay = SubstrateInterface(url="wss://shibuya-rpc.dwellir.com")
substrate_contract = SubstrateInterface(
    url='wss://rococo-contracts-rpc.polkadot.io')

In [16]:
def format_balance(amount: int):
    amount = format(
        amount / 10**substrate_relay.properties.get('tokenDecimals', 0),
        ".15g")
    return f"{amount} {substrate_relay.properties.get('tokenSymbol', 'UNIT')}"

In [17]:
def get_account_balance(account_address):
    result = substrate_relay.query("System", "Account", [account_address])
    balance = (result.value["data"]["free"] + result.value["data"]["reserved"])

    return format_balance(balance)

In [18]:
def get_polkadot_account_balance(account_address):
    """Method to get the balance of a Polkadot account."""
    result = substrate_relay.query("System", "Account", [account_address])
    balance = (result.value["data"]["free"] + result.value["data"]["reserved"])

    return format_balance(balance)

In [9]:
print(get_account_balance("5DvyRvNq5jpvjat2qkGhiKjJQdz5cwreJW5yxvLBLRpHnoGo"))

9.74665958012062 SBY


In [4]:
class GetAccountBalanceInput(BaseModel):
    """Inputs for get_account_balance"""

    account_address: str = Field(description="the address of the account to get the balance of")

class  GetAccountBalanceTool(BaseTool):
    name = "get_account_balance"
    description = """
        Useful when you want to get the balance of an polkadot account.
        The account address is the address of the account you want to get the balance of.
        The address format is ss58 encoded.
        """
    args_schema: Type[BaseModel] = GetAccountBalanceInput

    def _run(self, account_address: str):
        account_balance = get_account_balance(account_address)
        return account_balance

    def _arun(self, account_address: str):
        raise NotImplementedError("get_current_stock_price does not support async")
        
class GenerateInkPolkadotContractInput(BaseModel):
    """Inputs for generate_ink_polkadot_contract"""

    contract_description: str = Field(description="A description in simple english of what you would like the contract to do")

class GenerateInkPolkadotContractTool(BaseTool):
    name = "generate_ink_polkadot_contract"
    description = """
        Useful when you want to generate a polkadot contract in ink or just an ink contract.
        The contract description is a description of what you would like the contract to do.
        
        This also deploys the code to Shibuya Testnet.

        returns the contract address
        """
    args_schema: Type[BaseModel] = GenerateInkPolkadotContractInput

    def _run(self, contract_description: str):
        address = genCompileDeployContract(contract_description)
        return address

        
    def _arun(self, account_address: str):
        raise NotImplementedError("get_current_stock_price does not support async")


llm = ChatOpenAI(model="gpt-3.5-turbo-0613", temperature=0)
tools = [GetAccountBalanceTool(), GenerateInkPolkadotContractTool()]
agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)


In [20]:
embeddings = OpenAIEmbeddings()
db = DeepLake(
    dataset_path=f"hub://commanderastern/polka-code-3",
    read_only=True,
    embedding_function=embeddings,
)

Deep Lake Dataset in hub://commanderastern/polka-code-3 already exists, loading from the storage


In [21]:
retriever = db.as_retriever()
retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs["fetch_k"] = 20
retriever.search_kwargs["maximal_marginal_relevance"] = True
retriever.search_kwargs["k"] = 20

In [22]:
def filter(x):
    # filter based on source code
    if "something" in x["text"].data()["value"]:
        return False

    # filter based on path e.g. extension
    metadata = x["metadata"].data()["value"]
    return "only_this" in metadata["source"] or "also_that" in metadata["source"]

In [23]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

model = ChatOpenAI(model_name="gpt-3.5-turbo-16k")  # 'ada' 'gpt-3.5-turbo' 'gpt-4',
qa = ConversationalRetrievalChain.from_llm(model, retriever=retriever)

In [96]:
with open('code/lib.rs', 'w') as file:
    file.write(res)

In [11]:
import subprocess

def run_command(command):
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
    output, error = process.communicate()

    if process.returncode != 0:
        print(f"Error occurred: {error.decode().strip()}")
    else:
        print(f"Output: {output.decode().strip()}")


In [153]:
run_command("cd code && cargo contract build")

Error occurred:    Compiling my_contract v0.1.0 (C:\Users\asimj\AppData\Local\Temp\cargo-contract_7PqrsZ)
error: cannot find derive macro `SpreadAllocate` in this scope                  
 --> \\?\C:\Users\asimj\Documents\Polkadot-Seoul\langchain\code\lib.rs:7:14
  |
7 |     #[derive(SpreadAllocate)]
  |              ^^^^^^^^^^^^^^

error: could not compile `my_contract` due to preERROR: command ["\\\\?\\C:\\Users\\asimj\\.rustup\\toolchains\\1.69-x86_64-pc-windows-msvc\\bin\\cargo.exe", "build", "--color=always", "--target=wasm32-unknown-unknown", "-Zbuild-std=core,alloc", "--no-default-features", "--release", "--target-dir=c:\\Users\\asimj\\Documents\\Polkadot-Seoul\\langchain\\code\\target\\ink", "--features", "ink/ink-debug"] exited with code 101


In [12]:
menemonic = "edge lawn cost piano narrow annual movie duty forum patient check output"

In [101]:
# Upload WASM code
code = ContractCode.create_from_contract_files(
    metadata_file=os.path.join(os.getcwd(), 'code/target/ink',
                               'my_contract.json'),
    wasm_file=os.path.join(os.getcwd(), 'code/target/ink',
                           'my_contract.wasm'),
    substrate=substrate_relay)

# Deploy contract
print('Deploy contract...')
contract = code.deploy(keypair=Keypair.create_from_mnemonic(
    menemonic),
                       constructor="new",
                       args={'init_value': True},
                       value=0,
                       gas_limit={'ref_time': 25990000000, 'proof_size': 1199000},
                       upload_code=True)

print(f'✅ Deployed @ {contract.contract_address}')

Deploy contract...
✅ Deployed @ XXaWiLg4X1TGtSygr3RcU1YLGMsbLcsvgvLBqrc6jSZpgD5


In [ ]:
#setup for generating and deploy
embeddings = OpenAIEmbeddings()
db = DeepLake(
    dataset_path=f"hub://commanderastern/polka-code-2",
    read_only=True,
    embedding_function=embeddings,
)

retriever = db.as_retriever()
retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs["fetch_k"] = 20
retriever.search_kwargs["maximal_marginal_relevance"] = True
retriever.search_kwargs["k"] = 20

def filter(x):
    # filter based on source code
    if "something" in x["text"].data()["value"]:
        return False

    # filter based on path e.g. extension
    metadata = x["metadata"].data()["value"]
    return "only_this" in metadata["source"] or "also_that" in metadata["source"]
model_genContract = ChatOpenAI(model_name="gpt-3.5-turbo-16k", temprature = 0)  # 'ada' 'gpt-3.5-turbo' 'gpt-4',

qa = ConversationalRetrievalChain.from_llm(model_genContract, retriever=retriever)
menemonic = "edge lawn cost piano narrow annual movie duty forum patient check output"

def run_command(command):
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
    output, error = process.communicate()

    if process.returncode != 0:
        print(f"Error occurred: {error.decode().strip()}")
    else:
        print(f"Output: {output.decode().strip()}")

WARNING! temprature is not default parameter.
                    temprature was transferred to model_kwargs.
                    Please confirm that temprature is what you intended.


In [13]:
def genCompileDeployContract(description: str):
    # Generate contract
    questions = [
    f"""
    Give me a basic ink contract code, 
    Description of code = {description}
    """+"""
    -- There should be no print statments in the contract return everything.
    -- enclose the contract content in #startContract# and #endContract#
    -- follow this basic format when generating the code

    #startContract#
    #[ink::contract]
    mod contract_name \{

    }
    #endContract#
    """
    ]

    chat_history = []

    # extrating the contract code from the result
    result = qa({"question": questions[0], "chat_history": chat_history})
    chat_history.append((questions[0], result["answer"]))

    pattern = r'#startContract#(.*?)#endContract#'
    pattern2 = r'```rust(.*?)```'
    contract_code = re.search(pattern, result["answer"], re.DOTALL)
    res = ""
    
    if contract_code:
        res = contract_code.group(1).strip()
    else:
        contract_code = re.search(pattern2, result["answer"], re.DOTALL)
        if contract_code:
            res = contract_code.group(1).strip()
    res = r"""#![cfg_attr(not(feature = "std"), no_std, no_main)]""" + '\n'+ res
    post_process_code = re.sub(r'^\s*use.*\n?', '', res, flags=re.MULTILINE)

    post_process_code = re.sub(r'^\s*struct', 'pub struct', post_process_code, flags=re.MULTILINE)

    post_process_code = re.sub(r'^\s*#\s*\[derive\(.*\n?', '', post_process_code, flags=re.MULTILINE)

    print(post_process_code)

    # generating the constructor args
    new_function_pattern = r'(pub fn new.*?\))'
    new_function_match = re.search(new_function_pattern, post_process_code, re.DOTALL)

    if new_function_match:
        res = new_function_match.group(0)
    print(res)
    chat = ChatOpenAI()
    messages = [
        SystemMessage(content=r"""
        give the argumentsvalues for "pub fn new(value1: i32, value2: i32)" in the form of a dictionary
        -- Just the dictionary
        -- No need fore explanation or additional code
        -- empty dictionary is also fine
        -- for invalid input empty dictionary will be returned
        example: 
        Input: pub fn new(coolVal: i32)
        Output: {"coolVal": 1}"""),
        HumanMessage(content=f"{res}")
    ]
    constructor_args = ast.literal_eval(chat(messages).content)

    with open('code/lib.rs', 'w') as file:
        file.write(post_process_code)

    # compiling contract
    print(run_command("cd code && cargo contract build"))

    # Upload WASM code
    code = ContractCode.create_from_contract_files(
        metadata_file=os.path.join(os.getcwd(), 'code/target/ink',
                                'my_contract.json'),
        wasm_file=os.path.join(os.getcwd(), 'code/target/ink',
                            'my_contract.wasm'),
        substrate=substrate_relay)

    # Deploy contract
    print('Deploy contract...')
    contract = code.deploy(keypair=Keypair.create_from_mnemonic(
        menemonic),
                        constructor="new",
                        args=constructor_args,
                        value=0,
                        gas_limit={'ref_time': 25990000000, 'proof_size': 1199000},
                        upload_code=True)

    return contract.contract_address

In [208]:
genCompileDeployContract("A code that store value 1 and 2 and return the sum of the two values")

#![cfg_attr(not(feature = "std"), no_std, no_main)]
#[ink::contract]
mod contract_name {
    #[ink(storage)]
    pub struct Contract {
        value1: i32,
        value2: i32,
    }

    impl Contract {
        #[ink(constructor)]
        pub fn new() -> Self {
            Self { value1: 1, value2: 2 }
        }

        #[ink(message)]
        pub fn get_sum(&self) -> i32 {
            self.value1 + self.value2
        }
    }
}
pub fn new()
Output: [1/*] Building cargo project
 [2/4] Post processing code
 [3/4] Generating metadata
 [4/4] Generating bundle

Original wasm size: 36.6K, Optimized: 11.7K

The contract was built in DEBUG mode.

Your contract artifacts are ready. You can find them in:
c:\Users\asimj\Documents\Polkadot-Seoul\langchain\code\target\ink

  - my_contract.contract (code + metadata)
  - my_contract.wasm (the contract's code)
  - my_contract.json (the contract's metadata)
None
Deploy contract...
✅ Deployed @ aZYAzwJ36gkLCDtAWSHRfYBa995CErgyLVyZFATk3RDzvmT


In [24]:
agent.run(
    "I would like to build an ink contract and deploy it to Shibuya Testnet. It should be a contract that stores value 1 and 2 and return the sum of the two values"
)



> Entering new AgentExecutor chain...

Invoking: `generate_ink_polkadot_contract` with `{'contract_description': 'A contract that stores value 1 and 2 and returns the sum of the two values'}`


#![cfg_attr(not(feature = "std"), no_std, no_main)]
#[ink::contract]
mod contract_name {
    #[ink(storage)]
    pub struct Contract {
        value1: i32,
        value2: i32,
    }

    impl Contract {
        #[ink(constructor)]
        pub fn new() -> Self {
            Self {
                value1: 1,
                value2: 2,
            }
        }

        #[ink(message)]
        pub fn get_sum(&self) -> i32 {
            self.value1 + self.value2
        }
    }
}
pub fn new()
Output: [1/*] Building cargo project
 [2/4] Post processing code
 [3/4] Generating metadata
 [4/4] Generating bundle

Original wasm size: 36.6K, Optimized: 11.7K

The contract was built in DEBUG mode.

Your contract artifacts are ready. You can find them in:
c:\Users\asimj\Documents\Polkadot-Seoul\langchain\co

'I have generated an ink contract with the description you provided. The contract has been deployed to Shibuya Testnet and the contract address is `YUc12oBatBfu7rmNzn5Q8WKhfqZh2eoBNiQb45Z59MKpUpa`.'